# Data prep for analyses  

#### Notebook objectives:  
- Filter, clean and transform the raw AskDocs subreddit data
- Save the resulting analysis dataset  

#### Steps:  
First, we need to work out the needed data prep steps on a smaller data subset. We'll use the 2017 raw dataset for that.  
1. [Load 2017 data](#Load-2017-data)  
2. [Filter out irrelevant messages](#Filter-out-irrelevant-messages)  
3. [Consolidate post and cross-post content](#Consolidate-post-and-cross-post-content)
   - [Get cross-post subreddit names](#Get-cross-post-subreddit-names)  
4. [Consolidate the title and post body as the full user question](#Consolidate-the-title-and-post-body-as-the-full-user-question)
5. [Fix timestamp formats](#Fix-timestamp-formats)
6. [Update the selected fields list](#Update-the-selected-fields-list)

Next, we apply the above data prep steps to the entire dataset.  
7. [Load all data](#Load-all-data)  
8. [Apply data prep steps](#Apply-data-prep-steps)  
9. [Save the analysis dataset](#Save-the-analysis-dataset)
   

In [1]:
import pickle
import pandas as pd
import numpy as np
from IPython.display import display, HTML, Markdown, clear_output
import ipywidgets as widgets
import time 

In [3]:
DATA_PATH = 'data/'
OUTPUT_PATH = 'output/'

## Load 2017 data

In [32]:
with open(DATA_PATH + "reddit_askdocs_submissions_2017.pkl", "rb") as f:
    d_2017 = pickle.load(f)

In [143]:
df_2017 = pd.DataFrame(d_2017)

In [144]:
df_2017.head()

,author,author_flair_css_class,author_flair_text,brand_safe,can_mod_post,contest_mode,created_utc,domain,full_link,id,...,approved_at_utc,banned_at_utc,view_count,gilded,media_embed,secure_media_embed,author_created_utc,author_fullname,media,secure_media
0,[deleted],None,None,True,False,False,1514764452,self.AskDocs,https://www.reddit.com/r/AskDocs/comments/7nbw...,7nbwtn,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XenonCSGO,default,This user has not yet been verified.,True,False,False,1514764122,self.AskDocs,https://www.reddit.com/r/AskDocs/comments/7nbv...,7nbvsv,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[deleted],None,None,True,False,False,1514764055,self.AskDocs,https://www.reddit.com/r/AskDocs/comments/7nbv...,7nbvln,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DavisTheMagicSheep,default,This user has not yet been verified.,True,False,False,1514763799,self.AskDocs,https://www.reddit.com/r/AskDocs/comments/7nbu...,7nburb,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dontgetscooped,default,This user has not yet been verified.,True,False,False,1514763188,self.AskDocs,https://www.reddit.com/r/AskDocs/comments/7nbs...,7nbsw2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
df_2017['author_flair_text'].value_counts(dropna=False, normalize=True)

author_flair_text
This user has not yet been verified.    0.694449
None                                    0.305007
Medical Student                         0.000144
Nursing Student                         0.000112
Registered Nurse                        0.000048
Pharmacist                              0.000032
EMT                                     0.000016
Physician Assistant                     0.000016
Pharm.D. Student                        0.000016
Moderator                               0.000016
Physician                               0.000016
Biomedical Student                      0.000016
Web Developer                           0.000016
B.S., Medical Lab Sciences              0.000016
Nursing Graduate, RPN                   0.000016
Physician | Moderator                   0.000016
Psychologist                            0.000016
WEB DEVELOPER                           0.000016
Lead Moderator                          0.000016
Name: proportion, dtype: float64

## Filter out irrelevant messages  

Filtering out submissions not relevant to this analysis, like mod posts and banned posts. We want to analyze submissions that are medical questions from users. Mod posts are typically announcements, and bannded posts were deemed unrelated or inappropriate by subreddit moderators, so those types of posts can be filtered out as not relevant to analysis.

In [146]:
fname = 'reddit_submissions_selected_fields.csv'
selected_fields_df = pd.read_csv(fname)

In [147]:
display(Markdown(selected_fields_df[selected_fields_df['analysis_role'] == 'f'].to_markdown()))

|    | field_name    | type   | reddit_role   | analysis_role   | notes                                                                                                                     | type_long   | reddit_role_long                                                | analysis_role_long   |
|---:|:--------------|:-------|:--------------|:----------------|:--------------------------------------------------------------------------------------------------------------------------|:------------|:----------------------------------------------------------------|:---------------------|
| 14 | stickied      | b      | aa            | f               | Mods can pin up to 2 of their own posts to the top of the subreddit. This tag used to be called announcements.            | binary flag | author actions on the post (other than commenting)              | filtering            |
| 17 | banned_by     | c      | ma            | f               | The only values are NaN and moderators.                                                                                   | categorical | mod reactions to either post content or comments activity on it | filtering            |
| 21 | distinguished | c      | aa            | f               | Mods can tag posts as distinguished, usually used for subreddit management. Use this field for filtering out these posts. | categorical | author actions on the post (other than commenting)              | filtering            |

In [148]:
filter_fields = selected_fields_df[selected_fields_df['analysis_role'] == 'f']['field_name'].to_list()
filter_fields

['stickied', 'banned_by', 'distinguished']

In [149]:
def filter_submissions(df):
    original_rows_count = len(df)
    print(f'Original rows count: {original_rows_count}')
    
    filters = (df['stickied'] == True) | (df['distinguished'] == 'moderator') \
        | (df['banned_by'] == 'moderators')
    df = df.drop(df[filters].index)
    
    print(f'Filtered rows count: {len(df)}')
    print(f'% of rows filtered out: {(1-len(df)/original_rows_count)*100:.2f}')
    
    return df

In [150]:
df_2017 = filter_submissions(df_2017)

Original rows count: 62438
Filtered rows count: 59406
% of rows filtered out: 4.86


## Consolidate post and cross-post content

In [151]:
display(Markdown(selected_fields_df[selected_fields_df['analysis_role'] == 't']
                 .to_markdown()))

|    | field_name            | type   | reddit_role   | analysis_role   | notes                                                                                                        | type_long   | reddit_role_long   | analysis_role_long   |
|---:|:----------------------|:-------|:--------------|:----------------|:-------------------------------------------------------------------------------------------------------------|:------------|:-------------------|:---------------------|
| 19 | crosspost_parent      | s      | p             | t               | Cross-post parent post id.                                                                                   | short text  | post details       | transform            |
| 20 | crosspost_parent_list | l      | p             | t               | This ultimately contains the body text of a crossposted post. Just have to pull if out of the list of dicts. | long text   | post details       | transform            |

First, we need to clean up the `selftext` post body field a little.

In [152]:
df_2017['selftext'] = df_2017['selftext'].str.strip() 

In [153]:
df_2017['selftext'].str[:40].value_counts(dropna=False).head(20)

selftext
[deleted]                                         11833
[removed]                                          6109
                                                    253
NaN                                                  11
19, male, 180cm (5ft 11in), 128 pounds (             10
19, male, 180cm (5ft 11in), 137 pounds (              9
I'm not sure if this is the right place               7
* Age: 21\n* Sex: Female\n* Height: 5'10"\n           7
I lost my virginity last June to this ra              7
I don't know if this is the right place               6
Age: 18\n\nSex: Male\n\nDuration of Complain          6
* Age: 27\n* Sex: M\n* Height: 5'4"\n* Weig           6
Thanks for checking my post. I feel like              5
Hello for the past 50 days I have been d              5
Not sure if this is the right place to a              5
Age: 24\n\nSex: Female\n\nHeight: 5'4\n\nWeigh        5
I'm 1m64 (5'4). I stopped growing at 14.              5
Age: 18 \nSex: Male\nWeight: 90kg\nHeig

In [154]:
def clean_selftext_blanks(txt):
    if type(txt) == str:
        r = '' if txt in ['', '[deleted]', '[removed]'] else txt
    elif np.isnan(txt):
        r = ''
    else:
        r = txt
    return r

In [155]:
df_2017['selftext'] = df_2017['selftext'].apply(clean_selftext_blanks)

In [156]:
df_2017['selftext'].str[:40].value_counts(dropna=False).head(10)

selftext
                                                  18206
19, male, 180cm (5ft 11in), 128 pounds (             10
19, male, 180cm (5ft 11in), 137 pounds (              9
* Age: 21\n* Sex: Female\n* Height: 5'10"\n           7
I lost my virginity last June to this ra              7
I'm not sure if this is the right place               7
Age: 18\n\nSex: Male\n\nDuration of Complain          6
I don't know if this is the right place               6
* Age: 27\n* Sex: M\n* Height: 5'4"\n* Weig           6
Age: 24\n\nSex: Female\n\nHeight: 5'4\n\nWeigh        5
Name: count, dtype: int64

In [157]:
df_2017[df_2017['selftext'] == '']['crosspost_parent'].isna()\
    .value_counts(dropna=False).head(20)

crosspost_parent
True     18186
False       20
Name: count, dtype: int64

In [158]:
df_2017[df_2017['selftext'] != '']['crosspost_parent'].isna()\
    .value_counts(dropna=False).head(20)

crosspost_parent
True    41200
Name: count, dtype: int64

In [159]:
df_2017[df_2017['selftext'] == '']['crosspost_parent_list'].isna()\
    .value_counts(dropna=False).head(20)

crosspost_parent_list
True     18186
False       20
Name: count, dtype: int64

In [160]:
df_2017[df_2017['selftext'] != '']['crosspost_parent_list'].isna()\
    .value_counts(dropna=False).head(20)

crosspost_parent_list
True    41200
Name: count, dtype: int64

In [161]:
df_2017['crosspost_parent_list'].isna().value_counts(dropna=False)

crosspost_parent_list
True     59386
False       20
Name: count, dtype: int64

In [162]:
df_2017[df_2017['crosspost_parent_list'].notna()]['selftext'].str[:40]\
    .value_counts(dropna=False)

selftext
    20
Name: count, dtype: int64

After cleaning up the `selftext` field, the value counts above show that the crosspost content is available only when the cleaned `selftext` field is blank.

Now we can go ahead and use the crosspost content to fill in the blank post body content in the `selftext` field, if the crosspost content is available.

We can get the submission body and the crosspost subreddit from the dict inside the `crosspost_parent_list` field, all other relevant fields in the dict are duplicates of the submission-level fields. 

In [166]:
def get_crosspost_body(lst):
    # We'll take the body text of the first non-blank/non-deleted cross-post
    r = ''
    if type(lst) == list:
        for i in lst:
            txt = i['selftext'].strip()
            if txt.lower() not in ['', '[deleted]', '[removed]']:
                r = txt
                break
    return r

In [169]:
crosspost_fix_condition = (
    (df_2017['selftext'] == '') 
    & (df_2017['crosspost_parent_list'].notna())
)

df_2017['selftext'] = np.where(
    crosspost_fix_condition, 
    df_2017['crosspost_parent_list'].apply(get_crosspost_body), 
    df_2017['selftext']
)

In [170]:
df_2017[df_2017['crosspost_parent_list'].notna()]['selftext']

46      **If you have any ideas on how to fix this or ...
247     Hey guys.\n\nYesterday and today I had a massi...
553     Hello! I've always thought I've had pimples on...
1221    My mother is around 50 years old. So we've bee...
2227    Sorry for the low quality, it was hard to take...
2403    Last month, I had a visit to the doctor where ...
2758    (EDIT: Replace "amblyopia" with "strabismus," ...
2849    Hello, first--sorry for the length.\n\nHere's ...
3084    Last night I fell while snowboarding, like an ...
3156    I have been struggling for so long. A few mont...
3359                                                     
3567    Throw away account. I am posting this on other...
3618    I realize how rare this is but bear with me. L...
3901    Is it possible to have sleep apnea with out sn...
5126    I have read that certain malignancies or blood...
5307    **My short bio:** I'm an associate professor o...
5431    Not sure where to turn for this, my entire lif...
6736    Where 

### Get cross-post subreddit names  
The names of the cross-post subreddits could provide interesting info, so let's pull them out here into a variable as well.

In [175]:
def get_crosspost_subreddits(lst):
    r = ''
    if type(lst) == list:
        subreddits_list = []
        for i in lst:
            sr = i['subreddit']
            subreddits_list.append(sr)
            
        r = ', '.join(subreddits_list)
    
    return r

In [177]:
df_2017['crosspost_subreddits'] = df_2017['crosspost_parent_list']\
    .apply(get_crosspost_subreddits)

In [179]:
df_2017['crosspost_subreddits'].value_counts(dropna=False)

crosspost_subreddits
                    59386
medical                 3
Dermatology             2
optometry               2
SleepApnea              1
Drugs                   1
Allergies               1
needadvice              1
IAmA                    1
Medical_Students        1
RedditMD                1
sex                     1
MTHFR                   1
AskDoctor               1
STD                     1
DiagnoseMe              1
SkincareAddicts         1
Name: count, dtype: int64

## Consolidate the title and post body as the full user question  
Users sometimes ask all or part of their question in the title while the post body may be blank or may continue from the title.  
To get the full user question, we will combine the `title` and `selftext` fields into a single text string.  

In [180]:
df_2017['selftext'].str[:40].value_counts(dropna=False)

selftext
                                               18188
19, male, 180cm (5ft 11in), 128 pounds (          10
19, male, 180cm (5ft 11in), 137 pounds (           9
I lost my virginity last June to this ra           7
* Age: 21\n* Sex: Female\n* Height: 5'10"\n        7
                                               ...  
Sometimes 2-3 times a week, I have a str           1
I'm 32 and getting a circumcision tomorr           1
I am a 21 year old male. 6'2, 170lb. For           1
Basically it all started when I got hit            1
Okay so I don't have a picture but this            1
Name: count, Length: 40663, dtype: int64

In [182]:
df_2017[df_2017['selftext'] == '']['title'].str[:40].value_counts(dropna=False)

title
What is this?                               9
Scratched a bug bite on my scalp too har    7
Should I be worried?                        4
What bit me?                                4
Help                                        3
                                           ..
Sharp pain at base of neck and top of ba    1
my memory is terrible... is it worth see    1
I get deja vu's 5-6 times a day, it's no    1
Whey protein causing stomach issues         1
Strange cardiac symptoms                    1
Name: count, Length: 17897, dtype: int64

To make it easier to visually separate the title and post body in the combined text field, we'll use a pipe (`|`) separator.

In [185]:
df_2017['full_post_text'] = (
    df_2017['title'].str.strip() 
    + ' | '
    + df_2017['selftext'].str.strip()
)

In [187]:
df_2017['full_post_text'].str[:40].value_counts(dropna=False).head(20)

full_post_text
Scratched a bug bite on my scalp too har    9
What is this? |                             9
What bit me? |                              4
Neurological/Stroke Symptoms or Flashbac    4
Should I be worried? |                      4
Throat infection or something else?(pict    4
[26/M] Right knee problems for the last     4
I took a sip from a strangers water bott    3
OTC Antacids. Cimetidine vs Lansoprazole    3
My son is in a medically induced coma af    3
Picture: Why does the skin fold over my     3
How do I remove surgical stitches from t    3
Neck pain |                                 3
[NSFW] Did something really stupid last     3
Help |                                      3
Tonsil problems for 2 months |              3
My roommate won't stop coughing! What co    3
NSFW: Do these ridges in my stool look c    3
20M Been dealing with pain and burning f    3
What are these two marks on my abdomen?     3
Name: count, dtype: int64

In [189]:
(df_2017['full_post_text'] == '').value_counts()

full_post_text
False    59406
Name: count, dtype: int64

## Fix timestamp formats  

In [190]:
timestamp_fields = ['created_utc', 'edited']

In [191]:
df_2017['created_utc'].head()

0    1514764452
1    1514764122
2    1514764055
3    1514763799
4    1514763188
Name: created_utc, dtype: int64

In [193]:
pd.to_datetime(df_2017['created_utc']).head() # the default in pandas is 'ns'

0   1970-01-01 00:00:01.514764452
1   1970-01-01 00:00:01.514764122
2   1970-01-01 00:00:01.514764055
3   1970-01-01 00:00:01.514763799
4   1970-01-01 00:00:01.514763188
Name: created_utc, dtype: datetime64[ns]

In [192]:
pd.to_datetime(df_2017['created_utc'], unit='s').head()

0   2017-12-31 23:54:12
1   2017-12-31 23:48:42
2   2017-12-31 23:47:35
3   2017-12-31 23:43:19
4   2017-12-31 23:33:08
Name: created_utc, dtype: datetime64[ns]

In [195]:
pd.to_datetime(df_2017[df_2017['edited'].notna()]['edited']).head()

24   1970-01-01 00:00:01.514754516
25   1970-01-01 00:00:01.514754706
38   1970-01-01 00:00:01.514746962
48   1970-01-01 00:00:01.514742249
52   1970-01-01 00:00:01.514782799
Name: edited, dtype: datetime64[ns]

In [196]:
pd.to_datetime(df_2017[df_2017['edited'].notna()]['edited'], unit='s').head()

24   2017-12-31 21:08:36
25   2017-12-31 21:11:46
38   2017-12-31 19:02:42
48   2017-12-31 17:44:09
52   2018-01-01 04:59:59
Name: edited, dtype: datetime64[ns]

The raw timestamp fields are in epochs with seconds as units. The default units in pandas is 'ns'. The two fields appear to be in the same format, but the `edited` field is named less informatively.  
To make these timestamp fields easier to work with in pandas, let's do the following:  
- Convert them to 'ns' units, since that's the default unit in pandas. This way, we won't have to remember to explicitly specify the units each time we cast these fields to datetime in pandas.  
- Rename both fields to make it more clear that they are UTC timestamps stored in nanosecond epochs.  

In [198]:
df_2017['created_utc_ns_dt'] = df_2017['created_utc'] * 1_000_000_000

In [199]:
df_2017['edited_utc_ns_dt'] = df_2017['edited'] * 1_000_000_000

In [200]:
pd.to_datetime(df_2017['created_utc_ns_dt']).head()

0   2017-12-31 23:54:12
1   2017-12-31 23:48:42
2   2017-12-31 23:47:35
3   2017-12-31 23:43:19
4   2017-12-31 23:33:08
Name: created_utc_ns_dt, dtype: datetime64[ns]

In [203]:
pd.to_datetime(df_2017['edited_utc_ns_dt']).value_counts(dropna=False)

edited_utc_ns_dt
NaT                    51213
2017-01-23 23:21:01        2
2017-04-26 20:51:54        1
2017-04-28 13:05:22        1
2017-05-02 18:11:53        1
                       ...  
2017-08-20 21:31:34        1
2017-08-20 22:37:28        1
2017-08-20 22:41:12        1
2017-08-21 04:17:33        1
2017-01-01 04:09:38        1
Name: count, Length: 8193, dtype: int64

## Update the selected fields list  
Let's add the new fields we created for analyses and drop any fields we no longer need.

In [204]:
display(Markdown(selected_fields_df[~selected_fields_df['analysis_role'].isin(['f', 't'])].to_markdown()))

|    | field_name        | type   | reddit_role   | analysis_role   | notes                                                                                                                                                                                                                   | type_long   | reddit_role_long                                                | analysis_role_long   |
|---:|:------------------|:-------|:--------------|:----------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------|:----------------------------------------------------------------|:---------------------|
|  0 | author            | s      | a             | i               | Note the [deleted] and [removed] entries.                                                                                                                                                                               | short text  | author info                                                     | id                   |
|  1 | author_flair_text | c      | a             | a               | 31% NaN values.                                                                                                                                                                                                         | categorical | author info                                                     | analysis             |
|  2 | created_utc       | t      | p             | a               | Dups present.                                                                                                                                                                                                           | timestamp   | post details                                                    | analysis             |
|  3 | domain            | s      | p             | a               | Domain where the post originated from.                                                                                                                                                                                  | short text  | post details                                                    | analysis             |
|  4 | full_link         | u      | p             | r               | A link to the post on Reddit.                                                                                                                                                                                           | url         | post details                                                    | reference            |
|  5 | id                | s      | p             | i               | Post id                                                                                                                                                                                                                 | short text  | post details                                                    | id                   |
|  6 | locked            | b      | ma            | a               | Only 6 values are true, everything else false.                                                                                                                                                                          | binary flag | mod reactions to either post content or comments activity on it | analysis             |
|  7 | num_comments      | n      | ga            | a               | nan                                                                                                                                                                                                                     | numeric     | general subreddit users reactions to the post                   | analysis             |
|  8 | num_crossposts    | n      | aa            | a               | Both NaN and zeros present. Few values >0.                                                                                                                                                                              | numeric     | author actions on the post (other than commenting)              | analysis             |
|  9 | over_18           | b      | p             | a               | 98% false. Looks like a NSFW-type label on the post content.                                                                                                                                                            | binary flag | post details                                                    | analysis             |
| 10 | pinned            | b      | aa            | a               | Users can pin up to 4 posts to their profile.                                                                                                                                                                           | binary flag | author actions on the post (other than commenting)              | analysis             |
| 11 | score             | n      | ga            | a               | The score is based on up and down votes.                                                                                                                                                                                | numeric     | general subreddit users reactions to the post                   | analysis             |
| 12 | selftext          | l      | p             | a               | Can have [deleted] as values.                                                                                                                                                                                           | long text   | post details                                                    | analysis             |
| 13 | spoiler           | b      | aa            | a               | Spoiler tags are used to mark spoiler content, and they can blur the preview or thumbnails. Both mods and post authors can add a spoiler tag on a post. There were 30 true values in the sample, so decided to keep it. | binary flag | author actions on the post (other than commenting)              | analysis             |
| 15 | title             | l      | p             | a               | Title of the post, can be very long.                                                                                                                                                                                    | long text   | post details                                                    | analysis             |
| 16 | url               | u      | p             | r               | Url to the original post if crossposted or from other source.                                                                                                                                                           | url         | post details                                                    | reference            |
| 18 | edited            | t      | aa            | a               | 86% NaNs.                                                                                                                                                                                                               | timestamp   | author actions on the post (other than commenting)              | analysis             |
| 22 | author_fullname   | s      | a             | i               | Unclear what this is, and lots of NaNs, but decided to keep for now.                                                                                                                                                    | short text  | author info                                                     | id                   |

In [205]:
list(selected_fields_df[~selected_fields_df['analysis_role'].isin(['f', 't'])]['field_name'])

['author',
 'author_flair_text',
 'created_utc',
 'domain',
 'full_link',
 'id',
 'locked',
 'num_comments',
 'num_crossposts',
 'over_18',
 'pinned',
 'score',
 'selftext',
 'spoiler',
 'title',
 'url',
 'edited',
 'author_fullname']

In [206]:
# Final fields selection

selected_fields = [
    'author',
    'author_flair_text',
#    'created_utc', # replaced with the new created_utc_ns_dt field below
    'domain',
    'full_link',
    'id',
    'locked',
    'num_comments',
    'num_crossposts',
    'over_18',
#    'pinned',
    'score',
    'selftext', # edited to include the body of crossposted posts where needed
#    'spoiler',
    'title',
    'url',
#    'edited', # replaced with the edited_utc_ns_dt field below
#    'author_fullname',

    
    # newly added fields
    'crosspost_subreddits',
    'full_post_text',
    'created_utc_ns_dt',
    'edited_utc_ns_dt',
]

In [207]:
df_2017[selected_fields].head()

,author,author_flair_text,domain,full_link,id,locked,num_comments,num_crossposts,over_18,score,selftext,title,url,crosspost_subreddits,full_post_text,created_utc_ns_dt,edited_utc_ns_dt
0,[deleted],None,self.AskDocs,https://www.reddit.com/r/AskDocs/comments/7nbw...,7nbwtn,False,0,0.0,False,2,,Appendicitis removed 1 month ago but feel a pa...,https://www.reddit.com/r/AskDocs/comments/7nbw...,,Appendicitis removed 1 month ago but feel a pa...,1514764452000000000,NaN
1,XenonCSGO,This user has not yet been verified.,self.AskDocs,https://www.reddit.com/r/AskDocs/comments/7nbv...,7nbvsv,False,2,0.0,False,1,"So I've taken Pepcid AC, 10mg at morning and n...","1 Stopped taking Pepcid AC, now suffering symp...",https://www.reddit.com/r/AskDocs/comments/7nbv...,,"1 Stopped taking Pepcid AC, now suffering symp...",1514764122000000000,NaN
2,[deleted],None,self.AskDocs,https://www.reddit.com/r/AskDocs/comments/7nbv...,7nbvln,False,1,0.0,False,1,,My grandma has neck/back pain and little to no...,https://www.reddit.com/r/AskDocs/comments/7nbv...,,My grandma has neck/back pain and little to no...,1514764055000000000,NaN
3,DavisTheMagicSheep,This user has not yet been verified.,self.AskDocs,https://www.reddit.com/r/AskDocs/comments/7nbu...,7nburb,False,2,0.0,False,1,"I've had a cold for the last couple days now, ...",My ears feel like there is pressure inside of ...,https://www.reddit.com/r/AskDocs/comments/7nbu...,,My ears feel like there is pressure inside of ...,1514763799000000000,NaN
4,Dontgetscooped,This user has not yet been verified.,self.AskDocs,https://www.reddit.com/r/AskDocs/comments/7nbs...,7nbsw2,False,1,0.0,False,1,(first about me : 32 white male 5 foot 5 225lb...,IBS maybe?,https://www.reddit.com/r/AskDocs/comments/7nbs...,,IBS maybe? | (first about me : 32 white male 5...,1514763188000000000,NaN


## Load all data  

In [210]:
with open(DATA_PATH + "reddit_askdocs_submissions_2018_to_20220121.pkl", "rb") as f:
    d_ge2018 = pickle.load(f)

In [211]:
df = pd.concat(
    [
        df_2017, 
        pd.DataFrame(d_ge2018[2018]),
        pd.DataFrame(d_ge2018[2019]),
        pd.DataFrame(d_ge2018[2020]),
        pd.DataFrame(d_ge2018[2021]),
        pd.DataFrame(d_ge2018[2022])
    ]
)

In [212]:
del d_2017, df_2017, d_ge2018

In [214]:
df.head()

,author,author_flair_css_class,author_flair_text,brand_safe,can_mod_post,contest_mode,created_utc,domain,full_link,id,...,removed_by_category,updated_utc,steward_reports,og_description,og_title,removed_by,media_metadata,is_created_from_ads_ui,author_is_blocked,url_overridden_by_dest
0,[deleted],None,None,True,False,False,1514764452,self.AskDocs,https://www.reddit.com/r/AskDocs/comments/7nbw...,7nbwtn,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XenonCSGO,default,This user has not yet been verified.,True,False,False,1514764122,self.AskDocs,https://www.reddit.com/r/AskDocs/comments/7nbv...,7nbvsv,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[deleted],None,None,True,False,False,1514764055,self.AskDocs,https://www.reddit.com/r/AskDocs/comments/7nbv...,7nbvln,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DavisTheMagicSheep,default,This user has not yet been verified.,True,False,False,1514763799,self.AskDocs,https://www.reddit.com/r/AskDocs/comments/7nbu...,7nburb,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dontgetscooped,default,This user has not yet been verified.,True,False,False,1514763188,self.AskDocs,https://www.reddit.com/r/AskDocs/comments/7nbs...,7nbsw2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Apply data prep steps

1. Filter out irrelevant messages

In [215]:
df = filter_submissions(df)

Original rows count: 713040
Filtered rows count: 683202
% of rows filtered out: 4.18


2. Consolidate post and cross-post content
   

In [216]:
df['selftext'] = df['selftext'].str.strip() 

In [217]:
df['selftext'] = df['selftext'].apply(clean_selftext_blanks)

In [218]:
crosspost_fix_condition = (
    (df['selftext'] == '') 
    & (df['crosspost_parent_list'].notna())
)

df['selftext'] = np.where(
    crosspost_fix_condition, 
    df['crosspost_parent_list'].apply(get_crosspost_body), 
    df['selftext']
)

4. Get cross-post subreddit names

In [219]:
df['crosspost_subreddits'] = df['crosspost_parent_list']\
    .apply(get_crosspost_subreddits)

5. Consolidate the title and post body as the full user question

In [220]:
df['full_post_text'] = (
    df['title'].str.strip() 
    + ' | '
    + df['selftext'].str.strip()
)

In [221]:
# check that we don't have blanks in full_post_text
(df['full_post_text'] == '').value_counts()

full_post_text
False    683202
Name: count, dtype: int64

6. Fix timestamp formats

In [222]:
df['created_utc_ns_dt'] = df['created_utc'] * 1_000_000_000

In [223]:
df['edited_utc_ns_dt'] = df['edited'] * 1_000_000_000

7. Filter to the final selected fields

In [224]:
df = df[selected_fields]

In [225]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 683202 entries, 0 to 15884
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   author                683202 non-null  object 
 1   author_flair_text     447549 non-null  object 
 2   domain                683202 non-null  object 
 3   full_link             683202 non-null  object 
 4   id                    683202 non-null  object 
 5   locked                683202 non-null  bool   
 6   num_comments          683202 non-null  int64  
 7   num_crossposts        645295 non-null  float64
 8   over_18               683202 non-null  bool   
 9   score                 683202 non-null  int64  
 10  selftext              683202 non-null  object 
 11  title                 683202 non-null  object 
 12  url                   683202 non-null  object 
 13  crosspost_subreddits  683202 non-null  object 
 14  full_post_text        683202 non-null  object 
 15  create

## Save the analysis dataset

In [226]:
# Save file with on-the-fly compression
df.to_csv(
    DATA_PATH + 'reddit_askdocs_submissions_2017_to_20220121_analysis_ds.zip', 
    index=False
)